## Анастасия Куканова, A3200
### Задание 3. kNN

#### Реализация kNN

In [1]:
from sklearn import datasets
from sklearn.neighbors import KNeighborsClassifier
import numpy as np


class KNN(object):
    def __init__(self, k, points, classes, dist_func, weighted=False):
        self.k = k
        self.points = points
        self.classes = classes
        self.dist_func = dist_func
        self.weighted = weighted

    def classify(self, x):
        distances = []
        for i in range(len(self.points)):
            distances.append((i, self.dist_func(self.points[i], x)))
        distances.sort(key=lambda d: d[1])
        class_counts = {}
        k = self.k
        while k > 0:
            for j in range(self.k):
                cl = self.classes[distances[j][0]]
                with np.errstate(divide='ignore'):
                    w = 1 / distances[j][1] if self.weighted else 1
                if cl in class_counts:
                    class_counts[cl] += w
                else:
                    class_counts[cl] = w
            max_freq = max(list(map(lambda cl: class_counts[cl], class_counts)))
            candidates = list(filter(lambda cl: class_counts[cl] == max_freq, class_counts))
            if len(candidates) == 1:
                break
            else:
                k -= 1
        return candidates[0]

    def classify_many(self, xs):
        classes = list(map(self.classify, xs))
        return classes

#### Вспомогательный код

Для кросс-валидации я решила использовать алгоритм Leave One Out, так как датасет небольшой. Он будет считать точность:

In [2]:
def leave_one_out(x, y, error_func, model_builder, *args):
    cv = 0
    for i in range(len(x)):
        outer_x = np.array([x[i]])
        outer_y = np.array([y[i]])
        mask = np.ones(len(x), dtype=bool)
        mask[i] = 0
        new_x = x[mask]
        new_y = y[mask]
        model = model_builder(new_x, new_y, *args)
        cv += error_func(model(outer_x), outer_y)
    cv /= len(x)
    return cv

acc = lambda y_model, y_true: sum(list(map(lambda p: 1 if p[0] == p[1] else 0, zip(y_model, y_true)))) / len(y_model)

Нормализация датасета:

In [3]:
def normalize_dataset(x):
    for i in range(len(x[0])):
        x[:, i] /= max(x[:, i])

Однородные интерфейсы для моего kNN и kNN из scikit-learn:

In [4]:
def build_knn_model(x, y, k, p, weighted=False):
    # метрика Минковского
    lp = lambda p: (lambda x1, x2: sum(map(lambda x: abs(x) ** p, x1 - x2)) ** (1 / p))
    classifier = KNN(k, x, y, lp(p), weighted)
    return classifier.classify_many


def build_skl_knn(x, y, k, p, weighted=False):
    w = 'distance' if weighted else 'uniform'
    classifier = KNeighborsClassifier(k, p=p, weights=w)
    classifier.fit(x, y)
    return classifier.predict

In [5]:
def compare(x, y, cv_func, k_s, k_e, p, w=False):
    for k in range(k_s, k_e+1):
        print("k =", k)
        print("me:", leave_one_out(x, y, cv_func, build_knn_model, k, 1))
        print("sk:", leave_one_out(x, y, cv_func, build_skl_knn, k, 1))
        print()

#### Кросс-валидация и сравнение алгоритмов

Загрузка и нормализация датасета:

In [6]:
iris = datasets.load_iris()
__x__ = iris.data
__y__ = iris.target

normalize_dataset(__x__)

Манхэттенское расстояние (норма $l_1$), невзвешенный алгоритм, k от 5 до 10:

In [7]:
compare(__x__, __y__, acc, 5, 10, 1)

k = 5
me: 0.9533333333333334
sk: 0.9533333333333334

k = 6
me: 0.9533333333333334
sk: 0.9533333333333334

k = 7
me: 0.9466666666666667
sk: 0.9466666666666667

k = 8
me: 0.9533333333333334
sk: 0.9533333333333334

k = 9
me: 0.9533333333333334
sk: 0.9533333333333334

k = 10
me: 0.9533333333333334
sk: 0.9533333333333334



Манхэттенское расстояние (норма $l_1$), взвешенный алгоритм, k от 5 до 10:

In [8]:
compare(__x__, __y__, acc, 5, 10, 1, True)

k = 5
me: 0.9533333333333334
sk: 0.9533333333333334

k = 6
me: 0.9533333333333334
sk: 0.9533333333333334

k = 7
me: 0.9466666666666667
sk: 0.9466666666666667

k = 8
me: 0.9533333333333334
sk: 0.9533333333333334

k = 9
me: 0.9533333333333334
sk: 0.9533333333333334

k = 10
me: 0.9533333333333334
sk: 0.9533333333333334



Расстояние Евклида (норма $l_2$), невзвешенный алгоритм, k от 5 до 10:

In [9]:
compare(__x__, __y__, acc, 5, 10, 2)

k = 5
me: 0.9533333333333334
sk: 0.9533333333333334

k = 6
me: 0.9533333333333334
sk: 0.9533333333333334

k = 7
me: 0.9466666666666667
sk: 0.9466666666666667

k = 8
me: 0.9533333333333334
sk: 0.9533333333333334

k = 9
me: 0.9533333333333334
sk: 0.9533333333333334

k = 10
me: 0.9533333333333334
sk: 0.9533333333333334



Расстояние Евклида (норма $l_2$), взвешенный алгоритм, k от 5 до 10:

In [10]:
compare(__x__, __y__, acc, 5, 10, 2, True)

k = 5
me: 0.9533333333333334
sk: 0.9533333333333334

k = 6
me: 0.9533333333333334
sk: 0.9533333333333334

k = 7
me: 0.9466666666666667
sk: 0.9466666666666667

k = 8
me: 0.9533333333333334
sk: 0.9533333333333334

k = 9
me: 0.9533333333333334
sk: 0.9533333333333334

k = 10
me: 0.9533333333333334
sk: 0.9533333333333334



Таким образом, алгоритмы проявляют себя одинаково хорошо.

#### Grid search

Будем искать наилучшие параметры модели в следующих диапазонах:
* p (параметр метрики Минковского): от 1 до 5
* k: от 1 до 16

Так же будем сравнивать взвешенный и невзвешенный алгоритмы.

In [11]:
__acc__ = []
for __p__ in range(1, 6):
    for __k__ in range(1, 16):
        for __w__ in [False, True]:
            params = (__k__, __p__, __w__)
            __acc__.append((leave_one_out(__x__, __y__, acc, build_knn_model, *params), params))

max_acc = max(map(lambda x: x[0], __acc__))
print("Maximum accuracy:", max_acc)
best_params = list(filter(lambda x: x[0] == max_acc, __acc__))
for param_set in best_params:
    print("k =", param_set[1][0], ", p =", param_set[1][1], ", weighted =", param_set[1][2])

Maximum accuracy: 0.9666666666666667
k = 7 , p = 3 , weighted = False
k = 7 , p = 3 , weighted = True


Наилучший результат дал kNN, вычисляющий расстояния до 7 ближайших соседей по формуле $\rho(x,y) = \sqrt[3]{\sum\limits_{i=1}^{n}|x_i - y_i|^3}$ (вне зависимости от взвешенности алгоритма).

In [10]:
from random import shuffle
precision = lambda y_model, y_true: sum(list(map(lambda p: 1 if p[0] == p[1] else 0, zip(y_model, y_true)))) / len(y_model)

def k_fold(k, x, y, error_func, model_builder, *args, **kwargs):
    ind = list(range(len(x)))
    shuffle(ind)
    cv = 0
    q = int(len(x) / k)
    r = len(x) % k
    nums = []
    for i in range(r):
        nums.append(i * (q + 1))
    for i in range(r, k):
        nums.append(i * q + r)
    nums.append(len(x))
    for i in range(k):
        outer_x = []
        new_x = []
        outer_y = []
        new_y = []
        for j in range(len(x)):
            if j in range(nums[i], nums[i + 1]):
                outer_x.append(x[ind[j]])
                outer_y.append(y[ind[j]])
            else:
                new_x.append(x[ind[j]])
                new_y.append(y[ind[j]])
        new_x = np.array(new_x)
        new_y = np.array(new_y)
        outer_x = np.array(outer_x)
        outer_y = np.array(outer_y)
        model = model_builder(new_x, new_y, *args, **kwargs)
        cv += error_func(model(outer_x), outer_y)
    cv /= k
    return cv

In [11]:
__acc__ = []
for __p__ in range(1, 6):
    for __k__ in range(1, 16):
        for __w__ in [False, True]:
            params = (__k__, __p__, __w__)
            __acc__.append((k_fold(10, __x__, __y__, precision, build_knn_model, *params), params))

max_acc = max(map(lambda x: x[0], __acc__))
print("Maximum accuracy:", max_acc)
best_params = list(filter(lambda x: x[0] == max_acc, __acc__))
for param_set in best_params:
    print("k =", param_set[1][0], ", p =", param_set[1][1], ", weighted =", param_set[1][2])

Maximum accuracy: 0.9666666666666668
k = 7 , p = 2 , weighted = False
k = 8 , p = 2 , weighted = False
k = 2 , p = 3 , weighted = False
k = 6 , p = 3 , weighted = False
k = 15 , p = 4 , weighted = True
k = 3 , p = 5 , weighted = True
